In [ ]:
import json
from collections import defaultdict
from columnar import columnar
from statistics import mean, stdev
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
import math

In [ ]:
@dataclass
class PlayerStats:
    id_: str = ""
    elo: int = 1000
    wins: int = 0
    loses: int = 0
    last_pseudo: str = ""
    pseudos: set = field(default_factory=set)
    perf_history: list = field(default_factory=list)

    def update_elo(this, gain, opponant):
        perf = PerfHistory(opponant, this.elo, opponant.elo, gain)
        this.perf_history.append(perf)
        this.elo += gain

    def sets(this):
        return this.wins + this.loses

    def winrate(this):
        return this.wins / this.sets()

    def pic_elo(this):
        return max(this.elo_history())

    def worse_elo(this):
        return min(this.elo_history())

    def elo_history(this):
        return [p.my_elo for p in this.perf_history] + [this.elo]

    def plot_elo(this):
        plt.plot(this.elo_history())
        plt.show()

    def best_perf(this):
        return max(this.perf_history, key=lambda l: l.gain)

    def worse_perf(this):
        return min(this.perf_history, key=lambda l: l.gain)


@dataclass
class PerfHistory:
    opponant: PlayerStats
    my_elo: float
    his_elo: float
    gain: float


def update_ratings(player_stats_1, player_stats_2, winner, k):
    if player_stats_1 == winner:
        actual_outcome = 1
        loser = player_stats_2
    elif player_stats_2 == winner:
        actual_outcome = 0
        loser = player_stats_1
    else:
        raise Exception("Nobody won !")

    winner.wins += 1
    loser.loses += 1

    expected_outcome = 1 / \
        (1 + 10**((player_stats_2.elo - player_stats_1.elo) / 400))

    player_1_gain = k * (actual_outcome - expected_outcome)
    player_2_gain = k * ((1 - actual_outcome) - (1 - expected_outcome))
    player_stats_1.update_elo(player_1_gain, player_stats_2)
    player_stats_2.update_elo(player_2_gain, player_stats_1)

    error = (expected_outcome - actual_outcome) ** 2

    return error


In [ ]:
sets = json.load(open("data/all_sets.json", "r"))

In [ ]:
def compute_elo(sets_, k):
    errors = []
    players_stats = defaultdict(PlayerStats)

    for set_ in sets_:
        slots = set_["slots"]

        if len(slots) != 2:
            raise Exception("Not 2 slots")

        players = []
        lookup_entrant_player = {}
        for slot in slots:
            entrant = slot["entrant"]
            participants = entrant["participants"]

            if len(participants) != 1:
                raise Exception("Not 1 participant")

            participant = participants[0]
            entrant_id = entrant["id"]
            player_id = participant["player"]["id"]
            entrant_name = entrant["name"]

            lookup_entrant_player[entrant_id] = player_id
            player_stats = players_stats[player_id]
            
            player_stats.id_ = player_id
            player_stats.pseudos.add(entrant_name)
            player_stats.last_pseudo = entrant_name

            players.append(player_stats)

        winner = players_stats[lookup_entrant_player[set_["winnerId"]]]
        player_stats_1, player_stats_2 = players

        error = update_ratings(player_stats_1, player_stats_2, winner, k)
        errors.append(error)

    return players_stats, errors

In [ ]:
X = list(range(10, 400))
Y = []
Y2 = []
for i in X:
    players_stats, errors = compute_elo(sets, k=i)
    Y.append(mean(errors))
    Y2.append(stdev(errors))

minX = min(X, key=lambda i: Y[X.index(i)])
print(minX)

plt.plot(X,Y, label="Mean")
# plt.plot(X,Y2, label="Stdev")
plt.xlabel("K")
plt.ylabel("Error")
plt.legend()
plt.show()

In [ ]:
K = 50
MINIMAL_SETS_TO_BE_RANKED = 10

players_stats, errors = compute_elo(sets, k=K)

ranking = list(players_stats.values())
ranking = [p for p in ranking if p.sets() >= MINIMAL_SETS_TO_BE_RANKED]
ranking.sort(key=lambda player_stats: -player_stats.elo)

pure_elos = [player_stats.elo for player_stats in ranking]

data = []

for rank, player_stats in enumerate(ranking):
    best_perf = player_stats.best_perf()
    worse_perf = player_stats.worse_perf()
    data.append([
        rank+1,
        player_stats.last_pseudo,
        f"{player_stats.elo:.1f}",
        f"{player_stats.winrate()*100:.1f}%",
        player_stats.wins,
        player_stats.loses,
        player_stats.wins + player_stats.loses,
        f"{best_perf.my_elo:.1f} + {best_perf.gain:.1f}",
        f"{best_perf.opponant.last_pseudo} ({best_perf.his_elo:.1f})",
        f"{worse_perf.my_elo:.1f} + {worse_perf.gain:.1f}",
        f"{worse_perf.opponant.last_pseudo} ({worse_perf.his_elo:.1f})",
        f"{player_stats.pic_elo():.1f}",
        f"{player_stats.worse_elo():.1f}",
    ])

table = columnar(data, headers=["Rank", "Pseudo", "Elo", "WR", "W", "L", "S", "Best Perf", "vs", "Worse Perf", "vs", "Pic Elo", "Worse Elo"], justify=["r", "l","r", "r", "r", "r", "r", "r", "r", "r", "r", "r", "r"], no_borders=True, terminal_width=500)

# ranking[13].plot_elo()

print(table)
print()
print(f"General stats:")
print(f"Ranked player mean Elo : {mean(pure_elos):.2f}")
print(f"Ranked player Elo std : {stdev(pure_elos):.2f}")
print()
print(f"Methodology:")
print(f"- Elo is computed based on all Jura Smash tournament sets (from both single brackets and amateur, this allows to have more granularity for the bottom part of the ranking), for a total of {len(sets)} sets with {len(players_stats)} different players")
print(f"- To aggregate the results to players I used the the start.gg Id and not the participation name, to facilitate the reading the last name used in tournament is shown")
print(f"- Only the set final result is taken into account (win or lose, to be improved for future rankings)")
print(f"- Players with less than {MINIMAL_SETS_TO_BE_RANKED} don't aprear in the ranking, otherwise you appear in the ranking")
print(f"- Elo algorithm parameters: (K={K}, InitialElo={PlayerStats.elo})")
